### ----------------------------------------------------------------------------------------------------------
## 社会変革型 医療データサイエンティスト育成講座
# Chapter 7: ロジスティック回帰モデルと傾向スコア 配布用ノート
### ----------------------------------------------------------------------------------------------------------

#### 1. sklearnを用いたロジスティック回帰モデル構築

In [ ]:
# pima indiansデータの読み込み
# 前回の復習ですので、以下実行してみてください

import pandas as pd

# Jupyter Notebookのホームディレクトリに下記csvファイルを格納してください
filename = "diabetes.csv"

df = pd.read_csv(filename, sep=',',header=0)

# 平均値による欠損値の補完 
imputer_cols = df.columns[[1,2,3,4,5]]
correct_df = df.copy()
for i in imputer_cols:
    correct_df[i] = correct_df[i].mask(df[i]==0, df[i].mean())
    
correct_df.head()

In [ ]:
# (参考) BMIデータの確認
import matplotlib.pyplot as plt
%matplotlib inline

plt.hist(correct_df['BMI'],bins=20,edgecolor='k')
plt.xlabel('BMI')
plt.ylabel('count')
plt.title('BMI data from Pima Indians data')
plt.show()

In [ ]:
# データの前処理
# pandasのcut functionを使用します

######
# ここを消してください
######

In [ ]:
# scikit-learnによるモデル構築
import numpy as np
from sklearn.linear_model import LogisticRegression

# numpyのreshapeを使用して、説明変数を2次元配列に変換

######
# ここを消してください
######

# 正則化法の適用オプションとして、L1もしくはL2しか選択できません（defaultはL2）
# 正則化を行いたくない場合には、正則化項にかかるハイパーパラメータをゼロに近づける必要があります
# Cを大きな値とすることで、ハイパーパラメータが0に近づきます

######
# ここを消してください
######

# statsmodelsでのpenalty無しの場合とでは値が異なります
print(logisticModel.coef_[0])

# オッズ比を算出
print('odds ratio = ' + str(np.exp(logisticModel.coef_[0])))

In [ ]:
# BMIを連続値のまま説明変数として使用

######
# ここを消してください
######

print(logisticModel.coef_[0])
print('odds ratio = ' + str(np.exp(logisticModel.coef_[0])))

In [ ]:
# (参考) statsmodelsを使ったロジスティック回帰モデルの構築

# BMIをカテゴリデータ化
data_BMI_cat = pd.cut(correct_df['BMI'],[0,30,100],labels=[0,1])

# Outcomeを結合
data_BMI_cat = pd.concat([data_BMI_cat,correct_df['Outcome']],axis=1)
data_BMI_cat.head()

import statsmodels.formula.api as smf

# 説明変数と目的変数の関係を指定し、モデルを構築する
f = 'Outcome ~ BMI'
logitfit = smf.logit(formula=str(f), data=data_BMI_cat).fit()

print(logitfit.summary())

# パラメータは対数オッズ比を表すため、expの肩に乗せることでオッズ比を算出
print('odds ratio = ' + str(np.exp(logitfit.params[1])))

In [ ]:
# （参考） BMIを連続変数のまま、説明変数として使用するモデル

# BMIを説明変数として指定
f = 'Outcome ~ BMI'
logitfit = smf.logit(formula=str(f), data=correct_df).fit()

print(logitfit.summary())

#### 2. 混同行列、ROC曲線とAUC

In [ ]:
# まずsklearnを使用してモデル構築
# 復習ですので、以下実行してみてください

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# 説明変数の選択
X = correct_df[['Age','BMI']]
y = correct_df.loc[:, 'Outcome']

# テストデータを使用して評価を行うため、データを分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/4)

logisticModel = LogisticRegression()
logisticModel.fit(X_train, y_train)

In [ ]:
# (参考) 結果の可視化

# 散布図のための関数定義
def plot_scatter(df_pos,df_neg,cols=['Age','BMI'],dsize=10):
    plt.scatter(df_pos[cols[0]],df_pos[cols[1]],
                c='orange',
                s=dsize,
                label='positive',
               )
    plt.scatter(df_neg[cols[0]],df_neg[cols[1]],
                c='blue',
                s=dsize,
                label='negative',
                alpha=0.3, # そのままだとドットが重なって見にくいため、透明度を調整
               )

    plt.xlabel(cols[0])
    plt.ylabel(cols[1])

    # legendを可視化
    plt.legend()
    
# 変数定義
data_age = correct_df['Age']
data_BMI = correct_df['BMI']
df_pos = correct_df[correct_df['Outcome']==1]
df_neg = correct_df[correct_df['Outcome']==0]

# プロットのための座標を作成します
age = np.linspace(np.min(data_age),np.max(data_age),51)
BMI = np.linspace(np.min(data_BMI),np.max(data_BMI),51)

# age x BMI の２次元平面の各座標において、regressionで求めた関数から陽性率を計算します
prediction = np.zeros([51,51])
for i in range(51):
    for j in range(51):
        prediction[i,j] = logisticModel.predict_proba([[age[i],BMI[j]]])[0,1]

# 陽性率の関数とデータの散布図を同一グラフ上に描画
extent = [np.min(data_age), np.max(data_age), np.min(data_BMI), np.max(data_BMI)]
fig = plt.figure(figsize=[9,6])
ax = fig.add_subplot(1,1,1)
plot_scatter(df_pos,df_neg,dsize=15)
im = ax.imshow(prediction.T,
               extent=extent,
               origin='lower',
               cmap='pink'
              )
ax.set_xlabel('age')
ax.set_ylabel('BMI')
fig.colorbar(im)
plt.show()

In [ ]:
# 混同行列、classification_reportの確認
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

y_pred = logisticModel.predict(X_test)
labels = [1,0]
print(classification_report(y_test, y_pred, labels=labels))

# 混同行列
# 左上は真陽性(true positive)、右下は真陰性(true negative)、
# 左下は偽陽性(false positive)、右上は偽陰性(false negative)

######
# ここを消してください
######

In [ ]:
# ROC curve, およびarea under the curve (AUC) の計算
from sklearn.metrics import roc_curve, auc

# テストデータを使用してROC curveを計算するために、まずはテストデータの
# 各点における陽性率を、LogisticRegression.predict_proba関数を使用して計算します

######
# ここを消してください
######

# 次に、上で求めた真陽性率と、各点の真のOutcomeの値を使用して、
# 様々なthresholdsにおける真陽性率と偽陽性率を計算します

######
# ここを消してください
######

# ROC curveのarea under the curveを計算

######
# ここを消してください
######

# 実際にROC curveを描いてみます

######
# ここを消してください
######